# Intro to VADER Sentiment

In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ooshy\AppData\Roaming\nltk_data...


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")



In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



In [5]:
# Fetch all the news about Facebook Libra
libra_headlines = newsapi.get_everything(
    q="facebook AND libra",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Facebook Libra: {libra_headlines['totalResults']}")

# Show sample article
libra_headlines["articles"][0]



Total articles about Facebook Libra: 101


{'source': {'id': 'engadget', 'name': 'Engadget'},
 'author': 'Nicole Lee',
 'title': 'Facebook forms financial group to focus on payments',
 'description': 'Facebook has decided to delve deeper into commerce with the creation of a new group focused on payments. According to Bloomberg, it’s called F2 internally (short for Facebook Financial), and is meant to take care of all payments-related projects including Fac…',
 'url': 'https://www.engadget.com/facebook-financial-171810881.html',
 'urlToImage': 'https://o.aolcdn.com/images/dims?resize=1200%2C630&crop=1200%2C630%2C0%2C0&quality=95&image_uri=https%3A%2F%2Fs.yimg.com%2Fos%2Fcreatr-images%2F2018-12%2Fd50afa90-f8ac-11e8-9eff-b3c598cb4350&client=amp-blogside-v2&signature=bede3bdddfe4cfa405b6d7837243d8830c0e0da9',
 'publishedAt': '2020-08-10T17:18:10Z',
 'content': 'The company believes that enabling such purchases makes advertising becomes more valuable. CEO Mark Zuckerberg has said that he’s quite keen on growing in-app transactions. 

In [6]:
# Create the Facebook Libra sentiment scores DataFrame
libra_sentiments = []

for article in libra_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        libra_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
libra_df = pd.DataFrame(libra_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
libra_df = libra_df[cols]

libra_df.head()



,date,text,compound,positive,negative,neutral
0,2020-08-10,The company believes that enabling such purcha...,0.7992,0.219,0.000,0.781
1,2020-08-10,(Reuters) - Facebook Inc (FB.O) named former P...,-0.1027,0.000,0.045,0.955
2,2020-09-03,LONDON (Reuters) - Financial regulators must a...,0.2732,0.132,0.068,0.800
3,2020-09-03,FILE PHOTO: Bank of England Governor Andrew Ba...,0.0000,0.000,0.000,1.000
4,2020-08-12,Facebook Financial should help Facebook unify ...,0.4019,0.074,0.000,0.926


In [7]:
# Get descriptive stats from the DataFrame
libra_df.describe()



,compound,positive,negative,neutral
count,99.000000,99.000000,99.000000,99.000000
mean,0.229045,0.076222,0.023061,0.900737
std,0.328008,0.074431,0.041557,0.086439
min,-0.510600,0.000000,0.000000,0.659000
25%,0.000000,0.000000,0.000000,0.843000
50%,0.177900,0.071000,0.000000,0.920000
75%,0.484650,0.131500,0.043000,0.981000
max,0.885800,0.316000,0.173000,1.000000
